# KAN + MNIST + NTK
Purpose: Fit a KAN to the MNIST dataset, for benchmarking the KAN performance.

Furthermore, the PyTorch Lightning library is used for convenience.

In [1]:
# Imports
import numpy as np
import pandas as pd
import torch
from torch import nn
import lightning as L
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from lightning.pytorch.loggers import CSVLogger

import sys
sys.path.append('../convkans/kan_convolutional')
from KANLinear import *

# Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setup Randomness -- https://lightning.ai/docs/pytorch/stable/common/trainer.html
L.seed_everything(998244353, workers=True)

# CUDA Efficiency
torch.set_float32_matmul_precision('high')

Seed set to 998244353


In [2]:
# Dataset Setup -- Inspired by Hugo's Dataset Reformatting
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = MNIST("./temp/", train=True, download=True, transform=transform)
test_dataset = MNIST("./temp/", train=False, download=True, transform=transform)

# Reformatted, due to odd issues when using NTK on it
class LCDataset(Dataset): # Lightning Compatible Dataset
    def __init__(self, dataset, num_classes, limit=-1):
        self.limit = limit
        self.num_classes = num_classes
        if self.limit != -1:
            sub = list(np.random.permutation(np.arange(len(train_dataset)))[0:self.limit]) # Take a random sample of the first some elements.
            self.dataset = Subset(dataset, sub)
        else:
            self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        y_one_hot = torch.zeros(self.num_classes)
        y_one_hot[y] = 1
        return x, y_one_hot

batch_size = 64
ntk_loader = DataLoader(LCDataset(train_dataset, num_classes=10, limit=500), batch_size=batch_size, num_workers=10)
train_loader = DataLoader(LCDataset(train_dataset, num_classes=10), batch_size=batch_size, shuffle=True, num_workers=10)
test_loader = DataLoader(LCDataset(test_dataset, num_classes=10), batch_size=batch_size, num_workers=10)

In [3]:
# Model Declaration
class ClassicKAN(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.BatchNorm2d(1),
            nn.MaxPool2d(2),
            nn.Flatten(1),
            KAN(layers_hidden=[14*14, 64, 10], grid_size=2, spline_order=2)
        )
    
    def forward(self, x):
        return self.net(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred, y) # MSE Loss works better for NTK
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred, y) # MSE Loss works better for NTK
        v1 = torch.argmax(y_pred, dim=1)
        v2 = torch.argmax(y, dim=1)
        accuracy = torch.sum(torch.eq(v1, v2)) / len(y)
        self.log("test loss (Cross Entropy Loss)", loss)
        self.log("accuracy", accuracy)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-4, weight_decay=2e-3)
        return optimizer

In [4]:
# Train + Test + Results
model = ClassicKAN()
trained_model = L.Trainer(max_epochs=80, deterministic=True, logger=CSVLogger("logs", name="MNISTKANNTK"))
trained_model.fit(model, train_loader)
trained_model.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | net  | Sequential | 79.1 K | train
--------------------------------------------
79.1 K    Trainable params
0         Non-trainable params
79.1 K    Total params
0.316     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Training: |                                               | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=80` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃          Test metric           ┃          DataLoader 0          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│            accuracy            │       0.9743000268936157       │
│ test loss (Cross Entropy Loss) │      0.08256937563419342       │
└────────────────────────────────┴────────────────────────────────┘

[{'test loss (Cross Entropy Loss)': 0.08256937563419342,
  'accuracy': 0.9743000268936157}]

In [5]:
# Apply NTK
%run -i './introduction_code_modified.py'

def cross_entropy_loss_batch(y_hat, y):
    return F.cross_entropy(y_hat, y, reduction='none')

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
ntk_model = GaussianFit(model=model, device=device, noise_var=0.0)
ntk_model.fit(ntk_loader, optimizer, MSELoss_batch) # MSELoss seems to work better for NTK Models

In [6]:
def check_ntk_acc(model, dataloader):
    res = 0.0
    sumlength = 0
    model.eval()
    model.to(device)
    for it in iter(dataloader):
        x, y = it
        x = x.to(device)
        y = y.to(device)
        sumlength += len(x)
        res += (torch.argmax(model.forward(x), dim=1) == torch.argmax(y, dim=1)).sum()
    model.train()
    return res / sumlength

In [7]:
print(f'NTK Accuracy: {check_ntk_acc(ntk_model, test_loader)}')

NTK Accuracy: 0.5645999908447266


In [8]:
# R: True Value, C: Predicted Value
def make_predict_matrix(model, dataloader):
    res = np.zeros(shape=(10, 10), dtype=int)
    for it in iter(dataloader):
        x, y = it
        x = x.to(device)
        y = y.to(device)
        x_arg = torch.argmax(model.forward(x), dim=1)
        y_arg = torch.argmax(y, dim=1)
        for i in range(len(x_arg)):
            res[y_arg[i], x_arg[i]] += 1
    return res

In [9]:
print(make_predict_matrix(ntk_model, test_loader))

[[688  49  17  38  79  34  12  16  27  20]
 [142 760   6   1  14  94  31  12   6  69]
 [ 24  43 549  46 107  36  58  39  40  90]
 [ 49  45  52 415 110  45 144  46  64  40]
 [ 51  52  64  31 586  50  47  24  39  38]
 [ 74  48  87  40  60 430  39  44  43  27]
 [ 37  76  27  39  23  39 506 115  33  63]
 [ 28  20  24  25  49  46 271 483  33  49]
 [ 47  23  63  25  66  39  49  87 543  32]
 [ 80  76  63  28  23  31  85  26  44 553]]


In [10]:
print(make_predict_matrix(model, test_loader))

[[ 968    0    0    1    1    2    3    1    2    2]
 [   0 1126    2    2    0    2    2    0    1    0]
 [   5    0 1008    6    0    0    1    6    6    0]
 [   1    0    1  990    0    5    0    4    7    2]
 [   1    1    0    2  960    0    6    0    1   11]
 [   3    0    1   13    0  858    5    1    9    2]
 [   7    3    2    1    2    4  936    1    2    0]
 [   0    7   14    4    3    0    0  986    3   11]
 [   3    2    2   10    4    2    3    2  942    4]
 [   1    6    0   10    9    2    1    7    4  969]]
